# How-To, First Steps

## Load package and create basic options

In [2]:
import os
import sys
sys.path.insert(0, os.path.abspath('../'))

from robustportfolio.finance import make_option

Let's start with some basic options. Remember that 1D PutOnMax is just Put (the same goes for CallOnMax)

In [3]:
option1 = make_option(option_type='putonmax', strike=90)
option2 = make_option(option_type='putonmax', strike=80, payoff_dates = 5)
option3 = make_option(option_type='callonmax', strike=90, payoff_dates = [3,5])
option4 = make_option(option_type='put2call1') # American Put2Call1 option

Now let's see how these options got instantiated

In [4]:
print('Option 1: {}'.format(option1))
print('Option 2: {}'.format(option2))
print('Option 3: {}'.format(option3))
print('Option 4: {}'.format(option4))

Option 1: <robustportfolio.finance.derivatives.AmericanOption object at 0x10a307430>
Option 2: <robustportfolio.finance.derivatives.EuropeanOption object at 0x10a307d90>
Option 3: <robustportfolio.finance.derivatives.BermudanOption object at 0x114df6e20>
Option 4: <robustportfolio.finance.derivatives.AmericanOption object at 0x114df6ee0>


In [5]:
print('Option 1 payoff at x = 80, 90, and 100, t = 3: {}'.format(option1.payoff([[80], [90], [100]], 3)))
print('Option 2 payoff at x = 100, t = 3: {}'.format(option2.payoff(100, 3)))
print('Option 3 payoff at x = 100, t = 3: {}'.format(option3.payoff(100, 3)))
print('Option 3 payoff at x = 100, t = 4: {}'.format(option3.payoff(100, 4)))

Option 1 payoff at x = 80, 90, and 100, t = 3: [10.  0.  0.]
Option 2 payoff at x = 100, t = 3: [-inf]
Option 3 payoff at x = 100, t = 3: [10.]
Option 3 payoff at x = 100, t = 4: [-inf]


## Creating solver and solving some problems

Now let's create some basic 1D Problem with Rectangular multiplicative dynamics and no trading constraints. For that, we need module `robustportfolio.pricing`

In [7]:
from robustportfolio.pricing import *
import numpy as np
pm1 = Problem(starting_price=np.array(100), 
            price_dynamics=ConstantDynamics(support=RectangularHandler([.9, 1.1]), type='mult'),
            trading_constraints=NoConstraints, option=option1, 
            lattice=Lattice(delta=[1]), 
            time_horizon=5)

Now we create solver with some parameters.

Most of the time, there is no point in tweaking *all* of these parameters, only some, namely `enable_timer` and `iter_tick`

In [8]:
opts = {'convex_hull_filter': 'qhull', 'convex_hull_prune_fail_count': 0,
        'convex_hull_prune_success_count':0,'convex_hull_prune_corner_n': 3,'convex_hull_prune_seed': 0} 
solver = ConvhullSolver(enable_timer=True, pricer_options=opts, ignore_warnings=True, iter_tick=50)

Now we solve it and see the result $V_0(x_0)$

In [9]:
sol1 = solver.solve(pm1)
# the solution is simply a dictionary
print('Value: {0}'.format(sol1['Vf'][0][0]))

Precalculating points for value function evaluation: 0.0873 sec (CPU 0.0873 sec)
Computing value function in the last point: 0.0000 sec (CPU 0.0000 sec)
t = 4
iter = 77/90 (85.56%)
t = 3
t = 2
t = 1
iter = 4/23 (17.39%)
iter = 6/23 (26.09%)
t = 0
Computing value function in intermediate points in time: 0.4379 sec (CPU 0.4379 sec)
Solving the problem: 0.5257 sec (CPU 0.5257 sec)
Value: 5.327786420219619


Let's play around and change Lattice step

In [10]:
pm1.lattice = Lattice(delta=[.1])
sol2 = solver.solve(pm1)
print('Value: {0}'.format(sol2['Vf'][0][0]))

Precalculating points for value function evaluation: 0.6521 sec (CPU 0.6521 sec)
Computing value function in the last point: 0.0000 sec (CPU 0.0000 sec)
t = 4
iter = 91/818 (11.12%)
iter = 125/818 (15.28%)
iter = 186/818 (22.74%)
iter = 230/818 (28.12%)
iter = 282/818 (34.47%)
iter = 319/818 (39.00%)
iter = 352/818 (43.03%)
iter = 409/818 (50.00%)
iter = 470/818 (57.46%)
iter = 619/818 (75.67%)
iter = 697/818 (85.21%)
iter = 715/818 (87.41%)
iter = 730/818 (89.24%)
iter = 740/818 (90.46%)
iter = 742/818 (90.71%)
iter = 751/818 (91.81%)
t = 3
iter = 48/609 (7.88%)
iter = 55/609 (9.03%)
iter = 62/609 (10.18%)
iter = 78/609 (12.81%)
iter = 121/609 (19.87%)
iter = 150/609 (24.63%)
iter = 160/609 (26.27%)
iter = 187/609 (30.71%)
iter = 200/609 (32.84%)
iter = 208/609 (34.15%)
iter = 212/609 (34.81%)
iter = 262/609 (43.02%)
iter = 276/609 (45.32%)
iter = 289/609 (47.45%)
iter = 444/609 (72.91%)
iter = 455/609 (74.71%)
iter = 479/609 (78.65%)
iter = 481/609 (78.98%)
iter = 501/609 (82.27%)
it

Let's try 2D Problem with another option and *additive* dynamics.

In [ ]:
pm2 = Problem(starting_price=np.array([91,90]), price_dynamics=ConstantDynamics(support=RectangularHandler([[-1, 1],[-.75, 1]]), type='add'),
             trading_constraints=IdenticalMap(RealSpaceHandler()),
             option=option4,
             lattice=Lattice(delta=[.1,.1]), time_horizon=5)
solver.iter_tick = 200
sol3 = solver.solve(pm2)
print('Value: {0}'.format(sol3['Vf'][0][0]))

Precalculating points for value function evaluation: 0.8596 sec (CPU 0.8596 sec)
Computing value function in the last point: 0.0005 sec (CPU 0.0005 sec)
t = 4
iter = 951/5913 (16.08%)
iter = 998/5913 (16.88%)
iter = 1433/5913 (24.23%)
iter = 1690/5913 (28.58%)
iter = 1997/5913 (33.77%)
iter = 2005/5913 (33.91%)
iter = 3232/5913 (54.66%)
iter = 3444/5913 (58.24%)
iter = 3689/5913 (62.39%)
iter = 3854/5913 (65.18%)
iter = 3942/5913 (66.67%)
iter = 4376/5913 (74.01%)
iter = 4541/5913 (76.80%)
iter = 4638/5913 (78.44%)
iter = 4718/5913 (79.79%)
iter = 4744/5913 (80.23%)
iter = 4790/5913 (81.01%)
iter = 5110/5913 (86.42%)
iter = 5277/5913 (89.24%)
iter = 5420/5913 (91.66%)
iter = 5594/5913 (94.61%)
t = 3
iter = 660/3355 (19.67%)
iter = 805/3355 (23.99%)
iter = 937/3355 (27.93%)
